In [1]:
import json

import gc_utils
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import gridspec
from matplotlib.legend_handler import HandlerTuple

In [3]:
sim_dir = "/Users/z5114326/Documents/simulations/"
sim_lst = ["m12b", "m12c", "m12f", "m12i", "m12m"]

pub_data = sim_dir + "/snapshot_times_public.txt"
pub_snaps = pd.read_table(pub_data, comment="#", header=None, sep=r"\s+")
pub_snaps.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]
snp_lst = pub_snaps["index"].values
tim_lst = pub_snaps["time_Gyr"].values

In [ ]:
sim_dict = {sim: {} for sim in sim_lst}

for sim in sim_lst:
    grp_file = sim_dir + sim + "/" + "gc_groups.json"
    with open(grp_file, "r") as file:
        grp_dict = json.load(file)

    grp_lst = [int(grp) for grp in grp_dict[sim]]
    data_dict = {grp: {"all": [], "z0": []} for grp in grp_lst}

    ghost_file = sim_dir + sim + "/" + sim + "_ghosts.hdf5"
    ghost_data = h5py.File(ghost_file, "r")  # open processed data file

    for it_id in ghost_data.keys():
        src_dat = ghost_data[it_id]["source"]
        for grp in grp_lst:
            # if grp == 0:
            #     continue
            gmsk = np.abs(src_dat["grpid"][()]) == grp
            smsk = src_dat["s_flag"][()] == 1

            data_dict[grp]["all"].append(np.sum(gmsk))
            data_dict[grp]["z0"].append(np.sum(gmsk & smsk))

    sim_dict[sim] = data_dict